In [44]:
!pip install pettingzoo
!pip install "ray[all]" tensorflow "pettingzoo[all]"

Found existing installation: pyarrow 7.0.0
Uninstalling pyarrow-7.0.0:
  Successfully uninstalled pyarrow-7.0.0
  Using cached pyarrow-7.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached aiorwlock-1.3.0-py3-none-any.whl (10.0 kB)
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 kB)
  Using cached opencensus-0.11.0-py2.py3-none-any.whl (128 kB)
  Using cached starlette-0.23.1-py3-none-any.whl (64 kB)
  Using cached aiohttp-3.8.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached fastapi-0.89.1-py3-none-any.whl (55 kB)
  Using cached opentelemetry_exporter_otlp-1.1.0-py3-none-any.whl (7.3 kB)
  Using cached aiohttp_cors-0.7.0-py3-none-any.whl (27 kB)
  Using cached kubernetes-25.3.0-py2.py3-none-any.whl (1.4 MB)
  Using cached gpustat-1.0.0-py3-none-any.whl
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.1.0-py3-none-any.whl (14 kB)
  Using cached backoff-1.10.0-py2.py3-none-any.whl (31 kB)
  Using cached 

In [53]:
!pip install pygame
!pip install supersuit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.7 MB/s eta 0:00:00


In [38]:
import os

print(f"Number of CPUs in this system: {os.cpu_count()}")
import pyarrow

print(f"pyarrow: {pyarrow.__version__}")

Number of CPUs in this system: 16
pyarrow: 8.0.0


In [30]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

print(ray.cluster_resources())

2023-01-11 18:07:00,263	INFO worker.py:1538 -- Started a local Ray instance.


{'node:192.168.7.90': 1.0, 'accelerator_type:G': 1.0, 'object_store_memory': 7692727910.0, 'CPU': 16.0, 'memory': 15385455822.0, 'GPU': 1.0}


In [54]:
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from pettingzoo.classic import chess_v5
import supersuit as ss
import tensorflow as tf
from pettingzoo.classic.chess import chess

class CNNModel(TFModelV2):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        layer_state = tf.keras.layers.Input(shape=(8, 8, 8), name='state')
        conv_xs = tf.keras.layers.Conv2D(4, (1, 1), activation='relu')(layer_state)
        conv_s = tf.keras.layers.Conv2D(8, (2, 2), strides=(1, 1), activation='relu')(layer_state)
        conv_m = tf.keras.layers.Conv2D(12, (3, 3), strides=(2, 2), activation='relu')(layer_state)
        conv_l = tf.keras.layers.Conv2D(16, (4, 4), strides=(2, 2), activation='relu')(layer_state)
        conv_xl = tf.keras.layers.Conv2D(20, (8, 8), activation='relu')(layer_state)
        conv_rank = tf.keras.layers.Conv2D(3, (1, 8), activation='relu')(layer_state)
        conv_file = tf.keras.layers.Conv2D(3, (8, 1), activation='relu')(layer_state)

        f_xs = tf.keras.layers.Flatten()(conv_xs)
        f_s = tf.keras.layers.Flatten()(conv_s)
        f_m = tf.keras.layers.Flatten()(conv_m)
        f_l = tf.keras.layers.Flatten()(conv_l)
        f_xl = tf.keras.layers.Flatten()(conv_xl)
        f_r = tf.keras.layers.Flatten()(conv_rank)
        f_f = tf.keras.layers.Flatten()(conv_file)

        dense1 = tf.keras.layers.Concatenate(name='dense_bass')([f_xs, f_s, f_m, f_l, f_xl, f_r, f_f])
        dense2 = tf.keras.layers.Dense(256, activation='sigmoid')(dense1)
        dense3 = tf.keras.layers.Dense(128, activation='sigmoid')(dense2)
        dense4 = tf.keras.layers.Dense(56, activation='sigmoid')(dense3)
        dense5 = tf.keras.layers.Dense(64, activation='sigmoid')(dense4)
        dense6 = tf.keras.layers.Dense(32, activation='sigmoid')(dense5)

        value_head = tf.keras.layers.Dense(1)(dense6)

        self.base_model = tf.keras.Model(inputs=layer_state,
                           outputs=value_head)
        def forward(self, input_dict, state, seq_lens):
            model_out = self.base_model(input_dict['obs'].permute(0, 3, 1, 2))
        def value_function(self):
            ...